<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Trades-Endpoints/blob/main/Cancel_All_After_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 延遲全部撤單（Cancel All After）
# 倒數計時結束後，系統會撤銷目前所有掛單。此請求可以持續發送，以不斷延長“懲罰”倒數時間。

In [ ]:
# **限流：**1次/秒

In [ ]:
# 如果你的掛單數量很大，系統會分批撤銷，這個過程可能需要幾秒鐘。
# 此外，在“全部撤單”進行中的這段時間內，系統將會拒絕任何新的 ACTIVATE 和 CLOSE 請求。
# 等到全部撤單作業完成後，系統才會重新接受 ACTIVATE 和 CLOSE 請求。

In [ ]:
# POST /openApi/swap/v2/trade/cancelAllAfter

In [ ]:
import time                   # 導入時間模組，用於產生 timestamp
import requests               # 導入 requests 模組，發送 HTTP 請求
import hmac                   # 導入 hmac 模組，計算 HMAC 簽名
from hashlib import sha256    # 導入 sha256 雜湊，用於 HMAC 簽名

In [ ]:
APIURL = "https://open-api.bingx.com"   # BingX API 入口網址
APIKEY = ""        # 輸入你的 API KEY
SECRETKEY = ""     # 輸入你的 SECRET KEY

In [ ]:
def demo():
    payload = {}   # POST 請求內容，這裡沒有額外資料
    path = '/openApi/swap/v2/trade/cancelAllAfter'   # API 路徑：延遲全部撤單
    method = "POST"   # HTTP 請求方法
    paramsMap = {   # API 參數，依照官方文件填寫
        "type": "ACTIVATE",     # 操作型態（例如 ACTIVATE）
        "timeOut": 10           # 倒數秒數，倒數結束時撤銷所有掛單
    }
    paramsStr = parseParam(paramsMap)   # 將參數 dict 轉成查詢字串
    return send_request(method, path, paramsStr, payload)   # 發送 API 請求

In [ ]:
def get_sign(api_secret, payload):
    # 用 SECRET KEY 和查詢字串產生 HMAC-SHA256 簽名
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 輸出簽名（debug 用）
    return signature

In [ ]:
def send_request(method, path, urlpa, payload):
    # 拼接完整請求 URL，加上簽名
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 印出實際請求 URL（debug 用）
    headers = {
        'X-BX-APIKEY': APIKEY,    # APIKEY 放在 Header
    }
    # 發送 HTTP 請求
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text    # 回傳 API 回應內容（純文字）

In [ ]:
def parseParam(paramsMap):
    # 將參數 dict 按 key 排序，轉換成 URL 查詢字串
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])
    # 動態補上 timestamp
    if paramsStr != "":
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())   # 主程式測試，呼叫 demo()